In [1]:
import json
import numpy as np

In [2]:
friend_data = np.genfromtxt('grupee_data/friends.csv', delimiter=',', dtype=int)
friendships = [[] for _ in range(np.max(friend_data[:,0])+1)]

for id in set(friend_data[:,0]):
    friendships[id] = [id, friend_data[friend_data[:, 0] == id, 1]]

with open('grupee_data/preferences.json') as file:
    preferences = json.load(file)

with open('grupee_data/n_concerts.txt') as file:
    n_concerts = file.read().split('\n')[1:-1]
    concerts = np.array([(x.split(':')[0], x.split(':')[1]) for x in n_concerts])
    file.close()

In [3]:
friendship_matrix = np.empty((len(friendships), len(friendships)), dtype=int)

for i, id in enumerate(friendships):
    row = np.zeros(len(friendships))
    if id:
        row[id[1]] = 1
        friendship_matrix[i] = row
    else:
        friendship_matrix[i] = row
        
friendship_matrix = friendship_matrix + friendship_matrix.T - np.diag(friendship_matrix.diagonal())

<h3>Define likelihoods</h3>

In [4]:
visit_concerts = {
    'none': .2,
    'one': 1.8,
    'both': 39.3,
}

In [5]:
equalPrefs = []

for ind in range(len(preferences['0'])):
    genrePref = []
    for (id, row) in zip(preferences.keys(), preferences.values()):
        if row[ind] == '1': genrePref.append(int(id))
    equalPrefs.append(genrePref)

***
<h2>Start with Processing</h2>

<h3>Laplacian Matrix</h3>

In [66]:
degreeMat = np.diag(np.sum(friendship_matrix, axis=1))
laplacian = degreeMat - friendship_matrix

degrees = np.empty((len(friendships), 2))

for id in range(len(friendships)):
    degrees[id] = (id, laplacian[id][id])

print(f'12% of {len(friendships)}: {len(friendships) * .12}')
sortedDegrees = sorted(degrees, key=lambda tup: tup[1])
mostInfluential = np.array(sortedDegrees[len(friendships)-round(len(friendships) * .12):], dtype=int)
#np.savetxt('a_team_5.txt', np.array(list(reversed(mostInfluential[:,0]))), delimiter='\n', fmt='%i')

12% of 8311: 997.3199999999999


<h3>Some other model...<h3>

In [ ]:
!pip install networkx

In [46]:
import networkx as nx

graph = nx.Graph()

graph.add_nodes_from(range(len(friendship_matrix)))

for id, row in enumerate(friendship_matrix):
    edges = [(id, x) for x in np.where(row == 1)[0]]
    myPref = [i for i, x in enumerate(preferences[f'{id}']) if x == '1']
    weights = np.zeros((len(edges), 84))
    for friend in edges:
        weight = np.zeros(84)
        weight.fill(visit_concerts['none'])
        friendPref = [i for i, x in enumerate(preferences[f'{friend[1]}']) if x == '1']
        
        common_interest = set(myPref) & set(friendPref)
        my_exclusive = set(myPref) - set(friendPref)
        friend_exclusive = set(friendPref) - set(myPref)
        
        weight[list(common_interest)] = visit_concerts['both']
        weight[list(my_exclusive)] = visit_concerts['one']
        weight[list(friend_exclusive)] = visit_concerts['one']
    
    graph.add_edges_from(edges, weight=weight)